In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mysql.connector
import ast
from collections import Counter

In [3]:
raw_dataset = pd.read_csv('testing.csv', sep = '\t')

In [4]:
pd.set_option('display.max_columns', None)
#raw_dataset.head()

,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,tweet,language,mentions,urls,photos,replies_count,retweets_count,likes_count,hashtags,cashtags,link,retweet,quote_url,video,thumbnail,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest
0,1432131079658348544,1432131079658348544,2021-08-29 20:00:31 EDT,2021-08-29,20:00:31,-400,248245659,dancinglongrice,rara,NaN,pagi pacarku only https://t.co/RM621pwlXh,in,[],[],['https://pbs.twimg.com/tweet_video_thumb/E9-e...,0,0,0,[],[],https://twitter.com/dancinglongrice/status/143...,False,NaN,1,https://pbs.twimg.com/tweet_video_thumb/E9-e2q...,NaN,"38.0000,-97.0000,3000km",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
1,1432131079452905473,1432109282414235658,2021-08-29 20:00:31 EDT,2021-08-29,20:00:31,-400,120139457,kamorant,Karl Morant 🍁🍊🧡💪,NaN,"@vidman @brandan_eh @onstorm ""Ears Go Boomba S...",en,[],[],['https://pbs.twimg.com/tweet_video_thumb/E9_z...,0,0,0,[],[],https://twitter.com/kamorant/status/1432131079...,False,NaN,1,https://pbs.twimg.com/tweet_video_thumb/E9_znk...,NaN,"38.0000,-97.0000,3000km",NaN,NaN,NaN,NaN,"[{'screen_name': 'vidman', 'name': 'Vidman 📹 D...",NaN,NaN,NaN,NaN
2,1432131079230693377,1432131079230693377,2021-08-29 20:00:31 EDT,2021-08-29,20:00:31,-400,36776407,rejoiceamerica,RejoiceAmericaRadio,NaN,Now Playing Pressure into Praise by @LucindaMo...,en,"[{'screen_name': 'lucindamoore1', 'name': 'ms....",['http://www.rejoiceamericaradio.org'],['https://pbs.twimg.com/media/E9_zoJrXEAEtywD....,0,0,0,[],[],https://twitter.com/RejoiceAmerica/status/1432...,False,NaN,1,https://pbs.twimg.com/media/E9_zoJrXEAEtywD.jpg,NaN,"38.0000,-97.0000,3000km",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
3,1432131079209639941,1432131079209639941,2021-08-29 20:00:31 EDT,2021-08-29,20:00:31,-400,496100882,craazy__c,Chass🧚🏾,NaN,I just keep thinking about my bday trip in NYC...,en,[],[],[],0,0,0,[],[],https://twitter.com/Craazy__C/status/143213107...,False,NaN,0,NaN,NaN,"38.0000,-97.0000,3000km",NaN,NaN,NaN,NaN,[],NaN,NaN,NaN,NaN
4,1432131079192907781,1431978744256901121,2021-08-29 20:00:31 EDT,2021-08-29,20:00:31,-400,84173993,giocreo,Biagio,NaN,@ananananalong @eleytricaa naguevona como te d...,es,[],[],[],0,0,0,[],[],https://twitter.com/giocreo/status/14321310791...,False,NaN,0,NaN,NaN,"38.0000,-97.0000,3000km",NaN,NaN,NaN,NaN,"[{'screen_name': 'ananananalong', 'name': 'ᴀɴᴀ...",NaN,NaN,NaN,NaN


Build SQL DataBase

In [53]:
dataset = raw_dataset.loc[raw_dataset['hashtags'].notna(),:][['id', 'created_at','tweet', 'language','hashtags']]
dataset['hashtags'] = dataset['hashtags'].replace('[]', np.nan)
dataset = dataset.dropna()
dataset = dataset.reset_index() #dump old index into a column

,id,created_at,tweet,language,hashtags
28,1432131078169448455,2021-08-29 20:00:31 EDT,@thefreerifleman @GrampVictoria #prudent I lef...,en,['prudent']
34,1432131078043623426,2021-08-29 20:00:31 EDT,The young Marc Liegghio elated after hitting f...,en,['bombers']
36,1432131077993218053,2021-08-29 20:00:31 EDT,@donbellexz Premier link: https://t.co/RxLz2Q...,en,"['benandbenanddonbelle', 'upuanmvoutnow']"
43,1432131077674520578,2021-08-29 20:00:31 EDT,"Thinking of friends who are immigrants here, a...",en,['ida']
48,1432131077515141125,2021-08-29 20:00:31 EDT,WATCH: Dr @MariyaMoosajee highlights therapies...,en,"['ophthalmology', 'retina', 'ird']"


Get a connection to the database

In [138]:
tweet_db = mysql.connector.connect(host = 'localhost', user = 'root', passwd = 'cats', database = 'Tweet_hashtags')

Build Tweet Table

In [139]:
#insert data into tweet table
tweet_db.start_transaction() #start_transaction() modifies sql db
cursor = tweet_db.cursor()
try:
    for row_number in range(dataset.shape[0]):
        tweet_id = dataset['id'][row_number]
        tweet = dataset['tweet'][row_number]
        date_time = dataset['created_at'][row_number]
        language = dataset['language'][row_number]
        cursor.execute(
            "insert into tweets values(%s, %s, %s, %s)", 
            (int(tweet_id), tweet, date_time, language))
        
    # If you do an INSERT or UPDATE or DELETE, you have to commit the changes
    # If you want to test only and not save changes, change this to tweet_db.rollback()
    tweet_db.commit()
except:
    # The code here runs if there was an error.
    # If so, then we kill the connection, and make new connection
    cursor.close()
    tweet_db.close()
    tweet_db = mysql.connector.connect(host = 'localhost', user = 'root', passwd = 'cats', database = 'Tweet_hashtags')
    raise

Build Hashtag Table

In [140]:
#prepare unique hashtag list
raw_hashtags = dataset['hashtags'].unique()
hashtag_list = []
for hashtags in raw_hashtags:
    for index in range(len(ast.literal_eval(hashtags))):
        hashtag_list.append(ast.literal_eval(hashtags)[index])
hashtag_list = list(set(hashtag_list)) #set get the unique string from the list

#insert data into hashtag table
tweet_db.start_transaction()
cursor = tweet_db.cursor()
try:
    for index in range(len(hashtag_list)):
        hashtag_id = index
        hashtag = hashtag_list[index]
        cursor.execute("insert into hashtags values(%s, %s)", 
            (int(hashtag_id), hashtag))
    tweet_db.commit()
except:
    # The code here runs if there was an error.
    # If so, then we kill the connection, and make new connection
    cursor.close()
    tweet_db.close()
    tweet_db = mysql.connector.connect(host = 'localhost', user = 'root', passwd = 'cats', database = 'Tweet_hashtags')
    raise

Build Tweet_Hashtag Table

In [ ]:
#make df for table---tweet_hashtag
dataset_th = dataset[['id', 'hashtags']]
tweet_db.start_transaction()
cursor = tweet_db.cursor()
try:
    for row_number in range(dataset_th.shape[0]):
        hashtags_pertweet = dataset_th.loc[row_number, 'hashtags']
        hashtagslist_pertweet = ast.literal_eval(hashtags_pertweet)
        count_dict = Counter(hashtagslist_pertweet)
        tweet_id_pertweet = dataset_th.loc[row_number, 'id']
        
        for hashtag in set(hashtagslist_pertweet):
            cursor.execute("select hashtag_id from hashtags where hashtag = %s", 
                           (hashtag,))
            hashtag_id = cursor.fetchall()[0][0]
            
            count = count_dict[hashtag]
            
            cursor.execute("insert into tweet_hashtags values(%s, %s, %s)", 
                           (int(tweet_id_pertweet), hashtag_id, int(count)))
            #cursor.execute("insert into tweet_hashtags values(%s, %s)", 
                #(int(hashtag_id), hashtag))
    tweet_db.commit()
except:
    # The code here runs if there was an error.
    # If so, then we kill the connection, and make new connection
    cursor.close()
    tweet_db.close()
    tweet_db = mysql.connector.connect(host = 'localhost', user = 'root', passwd = 'cats', database = 'Tweet_hashtags')
    raise